In [1]:
!pip install stopwords textstat lexicalrichness

     |████████████████████████████████| 102kB 4.5MB/s 
     |████████████████████████████████| 1.9MB 33.5MB/s 
  Created wheel for lexicalrichness: filename=lexicalrichness-0.1.3-py2.py3-none-any.whl size=8741 sha256=4bf2632a4b9d0d19531d2f6d36bdf72da3bf8d09f05f471402b11e4da73ed7ff
  Stored in directory: /root/.cache/pip/wheels/ac/04/68/c48c4001edf1647b80b549897fd80082286ec253e343c73116
Successfully built lexicalrichness


In [2]:
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')   
stop_words = set(stopwords.words('english'))

from nltk.tag import pos_tag
from nltk import word_tokenize
from collections import Counter

import textstat
from lexicalrichness import LexicalRichness

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import pandas as pd
import os
import shutil

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
os.chdir("/content/drive/MyDrive/CS224N_Project")

In [ ]:
!wget https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/corona_fake.csv

--2021-03-14 04:50:35--  https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/corona_fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4501079 (4.3M) [text/plain]
Saving to: ‘corona_fake.csv’

corona_fake.csv     100%[===================>]   4.29M  --.-KB/s    in 0.1s    

2021-03-14 04:50:35 (37.1 MB/s) - ‘corona_fake.csv’ saved [4501079/4501079]



In [ ]:
shutil.move("corona_fake.csv", "data")

'data/corona_fake.csv'

In [4]:
pd.set_option('display.max_columns', 500)

df = pd.read_csv('data/corona_fake.csv')
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)

df.loc[5]['label'] = 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[242]['label'] = 'FAKE'

df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)

In [11]:
df.loc[df['label'] == 'FAKE', ['label']] = 1
df.loc[df['label'] == 'TRUE', ['label']] = 0

In [12]:
df

,title,text,source,label
0,missing,The 25th anniversary of Schengen seems more li...,Youtube,1
1,What should I do if I think I or my child may ...,First call your doctor or pediatrician for adv...,https://www.health.harvard.edu/,0
2,Rush Limbaugh makes obvious point that Wuhan c...,Long-time listeners of conservative talk radio...,https://www.naturalnews.com/,1
3,Anti-Vaxxers Already Undermining Potential Cor...,"For many, a COVID-19 vaccine can’t come soon e...",https://www.globalhealthnow.org/,0
4,Bill Gates’ Plan to Vaccinate the World,"In January of 2010, Bill and Melinda Gates ann...",https://www.activistpost.com/,1
...,...,...,...,...
1159,SOLVING THE WUHAN-5G COVID-19 MYSTERY,"You are about to swallow a giant red pill, my ...",https://www.lukestorey.com/,1
1160,"Coronavirus explained: Symptoms, lockdowns and...",The coronavirus pandemic has completely change...,https://www.cnet.com/,0
1161,COVID-19: DO DRACONIAN MEASURES SIGNIFY A MOVE...,Is anyone else buying the official dogma? Excu...,https://www.energytherapy.biz/,1
1162,missing,People have been trying to warn us about 5G fo...,Facebook,1


In [6]:
df.shape

(1164, 4)

In [8]:
(1164 / 10)


116.4

In [9]:
train = df[:931]
val = df[:931 + 116]
test = df[931 + 116:]

In [ ]:
t = df[df["label"] == "TRUE"]
f = df[df["label"] == "FAKE"]

In [ ]:
t.shape

(586, 4)

In [ ]:
f.shape

(578, 4)

In [10]:
train.to_csv("corona_train.csv")
val.to_csv("corona_val.csv")
test.to_csv("corona_test.csv")